In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_csv('apple_stock_1year_1h.csv', skiprows=1)

print("Available columns:", df.columns.tolist())

date_column = df.columns[0]
df['Datetime'] = pd.to_datetime(df[date_column])
df = df.set_index('Datetime')

# Look for High and Low price columns with different possible names
high_variants = ['high', 'High', 'HIGH']
low_variants = ['low', 'Low', 'LOW']

# Find which column names match our High/Low variants
high_col = None
for col in high_variants:
    if col in df.columns:
        high_col = col
        break

low_col = None
for col in low_variants:
    if col in df.columns:
        low_col = col
        break

if high_col and low_col:
    print(f"Using {high_col} and {low_col} for volatility calculation")
    df['Volatility'] = df[high_col] - df[low_col]
else:
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    print(f"Using {numeric_cols[0]} and {numeric_cols[1]}")
    df['Volatility'] = df[numeric_cols[0]] - df[numeric_cols[1]]

daily_volatility = df['Volatility'].resample('D').mean()


plt.figure(figsize=(12, 6))


plt.plot(daily_volatility.index, daily_volatility.values, color='red')


plt.plot(daily_volatility.index, daily_volatility.rolling(window=7).mean(), 
         color='red', linewidth=2, label='7-day Average')


plt.xlabel('Date')
plt.ylabel('Price Range ($)')
plt.title('Daily Price Volatility of Apple Stock')
plt.grid(True, alpha=0.3)
plt.legend()


plt.show()


print(f"Average daily volatility: ${daily_volatility.mean():.2f}")
print(f"Maximum daily volatility: ${daily_volatility.max():.2f} on {daily_volatility.idxmax().strftime('%Y-%m-%d')}")
